# Set Up: Jupyter Notebook

In [ ]:
# Import the required libraries
import requests
import pandas as pd

# ----------------- #

# Pandas extensions
from ingester3.extensions import *

# Set up for chache.
from ingester3.scratch import cache_manager

In [ ]:
# Check version of ingester3
from pkg_resources import get_distribution
installed_version = get_distribution('ingester3').version

if installed_version >= '1.8.1':
    print (f"You are running version {installed_version} which is consistent with the documentation")
else:
    print (f"""
You are running an obsolete version ({installed_version}). Run:
pip install ingester3 --upgrade 
to upgrade""")

# Download: Climate Data from CRU and ERA5 data sets
- Reasoning for variable choice
    - Document with more extensive reasoning for why a variable should or should not be included in bruno-gbl's repository: https://github.com/bruno-gbl/climate-data-country/blob/main/docs/Variables%20Overview.md
    - Document also includes information on the variables used in Garret's script to ingest climate variables at the pgm level.
    - Document also includes more detailed information about the specific coding of the different variables.
- CRU vs. ERA5 variables:
    - In this script, each climate data variable is either from the CRU or ERA5 climate data set. Both are available through the same World bank Climate Data portal.
    - However, their API and the time-span of available data differs slightly. Therefore, they are treated sligthly different before being merged into one dataset.
- Data Specifications on WorldBank Climate Data Platform:
    - CRU data set > timeseries > monthly > 1901-2022 > mean > model "ts4.07"
    - ERA5 data set > timeseries > monthly > 1950-2023 > mean > model "era5" > model label "x0.25"

#### Define Function: Download CRU data

In [2]:
# The API Structure:
# https://cckpapi.worldbank.org/cckp/v1/cru-x0.5_timeseries_{variable_name}_timeseries_monthly_1901-2022_mean_historical_cru_ts4.07_mean/all_countries?_format=json

# Function downloading the climate data from the World Bank Climate Knowledge Portal and turning it into a pandas DataFrame
def download_cru_data(cru_variable):
        
        # Define the URL for the JSON file
        url_climate_data = f"https://cckpapi.worldbank.org/cckp/v1/cru-x0.5_timeseries_{cru_variable}_timeseries_monthly_1901-2022_mean_historical_cru_ts4.07_mean/all_countries?_format=json"
        # Fetch the JSON data using a GET request
        response_climate_data = requests.get(url_climate_data)
    
        # Parse the JSON response into a Python dictionary, if the request was successful
        if response_climate_data.status_code == 200:
            climate_data_json = response_climate_data.json()
        else:
            print(f"Failed to fetch data.")
            climate_data_json = None
    
        # Flatten the nested structure into a list of dictionaries
        climate_data_json_only_data = climate_data_json["data"]

        rows = []

        for country, timeseries in climate_data_json_only_data.items():
            for date, value in timeseries.items():
                rows.append({"Country": country, "Date": date, "Value": value})
    
        # Convert the list into a pandas DataFrame
        climate_data_df = pd.DataFrame(rows)
    
        # Return the DataFrame for further use
        return climate_data_df


#### Download: CRU variables that should definitely be adopted.

In [3]:
# tas // Average Mean Surface Air Temperature // G: Only relative temperature variables
tas_cru_df = download_cru_data("tas")

# tasmax // Average Maximum Surface Air Temperature // G: Only relative temperature variables
tasmax_cru_df = download_cru_data("tasmax")

# tasmin // Average Minimum Surface Air Temperature // G: Only relative temperature variables
tasmin_cru_df = download_cru_data("tasmin")


#### Define Function: Download ERA5 data

In [4]:
# The API Structure:
# https://cckpapi.worldbank.org/cckp/v1/era5-x0.25_timeseries_{variable_name}_timeseries_monthly_1950-2023_mean_historical_era5_x0.25_mean/all_countries?_format=json

# Function downloading the climate data from the World Bank Climate Knowledge Portal and turning it into a pandas DataFrame
def download_era5_data(era5_variable):
        
        # Define the URL for the JSON file
        url_climate_data = f"https://cckpapi.worldbank.org/cckp/v1/era5-x0.25_timeseries_{era5_variable}_timeseries_monthly_1950-2023_mean_historical_era5_x0.25_mean/all_countries?_format=json"
        # Fetch the JSON data using a GET request
        response_climate_data = requests.get(url_climate_data)
    
        # Parse the JSON response into a Python dictionary, if the request was successful
        if response_climate_data.status_code == 200:
            climate_data_json = response_climate_data.json()
        else:
            print(f"Failed to fetch data.")
            climate_data_json = None
    
        # Flatten the nested structure into a list of dictionaries
        climate_data_json_only_data = climate_data_json["data"]

        rows = []

        for country, timeseries in climate_data_json_only_data.items():
            for date, value in timeseries.items():
                rows.append({"Country": country, "Date": date, "Value": value})
    
        # Convert the list into a pandas DataFrame
        climate_data_df = pd.DataFrame(rows)
    
        # Return the DataFrame for further use
        return climate_data_df

#### Download: ERA5 variables that should definitely be adopted.

In [5]:
# cdd65 // Cooling Degree Days (ref-65°F)
cdd65_era5_df = download_era5_data("cdd65")

# hd35 // Number of Hot Days (Tmax > 35°C)
hd35_era5_df = download_era5_data("hd35")

# hd40 // Number of Hot Days (Tmax > 40°C)
hd40_era5_df = download_era5_data("hd40")

# hd42 // Number of Hot Days (Tmax > 42°C)
hd42_era5_df = download_era5_data("hd42")

# hdd65 // Heating degree days (ref-65°F)
hdd65_era5_df = download_era5_data("hdd65")

# hi35 // Number of Days with Heat Index > 35°C
hi35_era5_df = download_era5_data("hi35")

# hi37 // Number of Days with Heat Index > 37°C
hi37_era5_df = download_era5_data("hi37")

# hurs // Relative Humidity
hurs_era5_df = download_era5_data("hurs")

# prpercnt // Precipitation Percent Change
prpercnt_era5_df = download_era5_data("prpercnt")

# rx1day // Average Largest 1-Day Precipitation
rx1day_era5_df = download_era5_data("rx1day")

# rx5day // Average Largest 5-Day Cumulative Precipitation
rx5day_era5_df = download_era5_data("rx5day")

# tnn // Minimum of Daily Min-Temperature
tnn_era5_df = download_era5_data("tnn")

# txx // Maximum of Daily Max-Temperature
txx_era5_df = download_era5_data("txx")


#### Download: ERA5 variables that should probably be adopted.

In [6]:
# pr // Precipitation // G: "Total wet day precipitation" // similar, not identical to existing "pr" variable // This variable is average precipitation over a given time // Potentially also includes smaller variation of rain below 1mm
pr_era5_df = download_era5_data("pr")

# hd30 // Number of Hot Days (Tmax > 30°C) //  G: - // Overlaping with existing hd35, hd40, hd42
hd30_era5_df = download_era5_data("hd30")

# hd50 // Number of Hot Days (Tmax > 50°C) // G: - // Only daily maximum temperature per month/year.
hd50_era5_df = download_era5_data("hd50")

# hi39 // Number of Days with Heat Index > 39°C // Did not include variable that was available
hi39_era5_df = download_era5_data("hi39")

# hi41 // Number of Days with Heat Index > 41°C // G: - // Did not include variable that was available
hi41_era5_df = download_era5_data("hi41")

# r50mm // Number of Days with Precipitation >50mm // G: - // Only "Very heavy precipitation days" = days with more than 20mm
r50mm_era5_df = download_era5_data("r50mm")

# r95ptot // Precipitation amount during wettest days //  G: - // Did not include identical variable "Very wet day precipitation"
r95ptot_era5_df = download_era5_data("r95ptot")

# tr23 // Number of Tropical Nights (T-min > 23°C) // [Very similar to "tr"]
tr23_era5_df = download_era5_data("tr23")

# tr26 // Number of Tropical Nights (T-min > 26°C) // [Very similar to "tr"]
tr26_era5_df = download_era5_data("tr26")

# tr29 // Number of Tropical Nights (T-min > 29°C) // [Very similar to "tr"]
tr29_era5_df = download_era5_data("tr29")

# tr32 // Number of Tropical Nights (T-min > 32°C) // [Very similar to "tr"]
tr32_era5_df = download_era5_data("tr32")


#### Download: ERA5 variables that should probably NOT be adopted.

In [7]:
# fd // Number of Frost Days (Tmin < 0°C) // G: Almost identical variable // G: count variable // Here: Average over time, i.e. data period. Smoothed-out, long-term perspective of Frost Days; Less interesting for forecasting.
fd_era5_df = download_era5_data("fd")

# id // Number of Ice Days (Tmax < 0°C) // G: Almost identical variable // G: count variable // Here: Average over time, i.e. data period. Smoothed-out, long-term perspective of Ice Days; Less interesting for forecasting.
id_era5_df = download_era5_data("id")


# Data Wrangling: Preparing for ingestion

#### Turning all variable-specific dataframes into one dictionary of dataframes
- Easier filtering for which variables to include in the ingestion.
- Simplified code.

In [8]:
# Creating a dictionary of dataframes with the respective variable names as labels, to be able name the value columns according to the variable name.
dataframes_dict = {
    # CRU variables that should definitely be adopted.
    "climate_tas": tas_cru_df,
    "climate_tasmax": tasmax_cru_df,
    "climate_tasmin": tasmin_cru_df,
    # ERA5 variables that should definitely be adopted.
    "climate_cdd65": cdd65_era5_df,
    "climate_hd35": hd35_era5_df,
    "climate_hd40": hd40_era5_df,
    "climate_hd42": hd42_era5_df,
    "climate_hdd65": hdd65_era5_df,
    "climate_hi35": hi35_era5_df,
    "climate_hi37": hi37_era5_df,
    "climate_hurs": hurs_era5_df,
    "climate_prpercnt": prpercnt_era5_df,
    "climate_rx1day": rx1day_era5_df,
    "climate_rx5day": rx5day_era5_df,
    "climate_tnn": tnn_era5_df,
    "climate_txx": txx_era5_df,
    # ERA5 variables that should probably be adopted.
    "climate_pr": pr_era5_df,
    "climate_hd30": hd30_era5_df,
    "climate_hd50": hd50_era5_df,
    "climate_hi39": hi39_era5_df,
    "climate_hi41": hi41_era5_df,
    "climate_r50mm": r50mm_era5_df,
    "climate_r95ptot": r95ptot_era5_df,
    "climate_tr23": tr23_era5_df,
    "climate_tr26": tr26_era5_df,
    "climate_tr29": tr29_era5_df,
    "climate_tr32": tr32_era5_df,
    # ERA5 variables that should probably NOT be adopted.
    "climate_fd": fd_era5_df,
    "climate_id": id_era5_df,
}

# Delete the standalone variable-specific dataframes to avoid confusion
for key in dataframes_dict.keys():
    cru_var = key.replace("climate_", "") + "_cru_df"
    era5_var = key.replace("climate_", "") + "_era5_df"
    if cru_var in globals():
        del globals()[cru_var]
    if era5_var in globals():
        del globals()[era5_var]

# Delete unnecessary objects after the loop finishes
del key
del cru_var
del era5_var

#### Bring all dataframes in the dictionary to the same time period: Start from January 1990
- Convert the "Date" column to a datetime object
- Filter the data to only include data from January 1990 onwards

In [9]:
# Create a function to filter the time period to only include data from January 1990 onwards
def filter_timeseries_1990(df):
    # Convert the "Date" column to a datetime object
    df["Date"] = pd.to_datetime(df["Date"])
    # Filter the DataFrame to only include data from January 1990 onwards
    return df[df["Date"] >= "1990-01-01"]

# Apply filtering to all DataFrames in the dictionary
dataframes_dict = {key: filter_timeseries_1990(df) for key, df in dataframes_dict.items()}

#### Merge all variables from the dictionary into one dataframe

In [10]:
# Function to merge dataframes from a dictionary into a single DataFrame
def merge_datasets_from_dict(dataframes):
    # Initialize an empty DataFrame to start merging
    merged_df = pd.DataFrame()

    # Iterate through the dictionary
    for var_name, df in dataframes.items():
        # Check if the DataFrame has the expected structure
        if set(df.columns) != {"Country", "Date", "Value"}:
            raise ValueError(f"DataFrame for '{var_name}' does not have the expected columns: {df.columns}")
        
        # Rename the 'Value' column to the variable name
        df = df.rename(columns={"Value": var_name})

        # Merge with the existing DataFrame
        if merged_df.empty:
            merged_df = df
        else:
            merged_df = pd.merge(merged_df, df, on=["Country", "Date"], how="outer")
    
    return merged_df

# Execute function and merge all DataFrames
merged_df = merge_datasets_from_dict(dataframes_dict)

# Clean up workspace by deleting the non-relevant DataFrames
climate_variables_df = merged_df
del merged_df
del dataframes_dict

## Data Wrangling: Turn merged data set of all climate variables into a suitable VIEWS format
- Change Country Codes
- Change Month Codes
- Commands based on ingester3


#### Wrangling: Inserting month_id column

In [ ]:
# Adapt the data to VIEWS format using the Ingester3 extensions
climate_variables_df = pd.DataFrame.cm.from_datetime(climate_variables_df, datetime_col='Date')

#### Wrangling: Archiving dataframe before c_id column

In [60]:
# Archive: climate_variables_df before creating the c_id column
climate_variables_df_pre_sv = climate_variables_df
del climate_variables_df


### Wrangling: c_id approach 1
- Created dataframe: climate_variables_df_c_id_1
- Problem
    - ! De facto inaccurate. Treats Sudan (SDN) to represent all of Sudan before split in 2011. However, climate data for country SDN is for today's borders of Sudan, i.e. Northern Sudan, for all months in the data.
    - ! For further information and potential solutions, see further down.

In [ ]:
# Soft validation: Create valid_id and assign value "true" only to the country_months that resonate with the VIEWS database.
    
climate_variables_df_c_id_1 = pd.DataFrame.cm.soft_validate_iso(climate_variables_df_pre_sv, iso_col='Country', month_col='month_id')

# Filter country-months for valid_id == True
climate_variables_df_c_id_1 = climate_variables_df_c_id_1[climate_variables_df_c_id_1.valid_id==True]

# Create the c_id column
climate_variables_df_c_id_1 = pd.DataFrame.cm.from_iso(df=climate_variables_df_c_id_1, iso_col='Country', month_col='month_id')


#### Wranglin: c_id approach 2
- If an iso3-code represents different geographical areas at different points in time, we only keep the most recent.
- This ensures that data that represents a specific area today is not used as data for a country with a identical iso3-code but a different geographical area.

- Obvious example: Sudan (SDN)
    - In the data, teh coutnry code SDN gives historical data for the region of today's Sudan.
    - For July 2011 onwards, ingester3 codes this correctly as c_id "245".
    - However, for pre-July 2011, ingester3 codes the country-months as having the c_id "59", which represents all of Sudan.
    - The problem stems from Sudan changing shape in 2011, but keeping the same ISO3-Code "SDN".

In [61]:
# Step 1: Sort the data by 'Country', 'c_id', and 'month_id' to ensure the latest month is at the end for each combination
climate_variables_df_c_id_1 = climate_variables_df_c_id_1.sort_values(by=['Country', 'c_id', 'month_id'], ascending=[True, True, True])

# Step 2: Identify the latest 'c_id' for each 'Country'
latest_cid_per_country = climate_variables_df_c_id_1.groupby('Country').tail(1)['c_id']

# Step 3: Filter the dataset to keep only the rows with the latest 'c_id' for each 'Country'
climate_variables_df_c_id_2 = climate_variables_df_c_id_1[climate_variables_df_c_id_1['c_id'].isin(latest_cid_per_country)]

# Step 4: Verify the filtered dataset
print(climate_variables_df_c_id_1)


       Country       Date  climate_tas  climate_tasmax  climate_tasmin  \
396        AFG 1990-01-01         0.44            5.30           -4.41   
397        AFG 1990-02-01         2.15            7.32           -3.00   
398        AFG 1990-03-01         6.85           13.49            0.23   
399        AFG 1990-04-01        13.09           19.79            6.44   
400        AFG 1990-05-01        20.00           27.83           12.22   
...        ...        ...          ...             ...             ...   
100363     ZWE 2023-08-01          NaN             NaN             NaN   
100364     ZWE 2023-09-01          NaN             NaN             NaN   
100365     ZWE 2023-10-01          NaN             NaN             NaN   
100366     ZWE 2023-11-01          NaN             NaN             NaN   
100367     ZWE 2023-12-01          NaN             NaN             NaN   

        climate_cdd65  climate_hd35  climate_hd40  climate_hd42  \
396              0.00          0.00         

--------------------------

# Problem: Testing how country data is structured in the original data. Potential issue with c_id column.
- Basically: Does the "SDN" data before 2011 only represent data from Northern Sudan in its current borders, or does it represent all of Sudan?
- Same question relates to other countries with territorial changes.

In [ ]:
# Checking precipitation, as there is probably a bigger difference between the two countries.

# Filter data for Sudan pre- and post-July 2011
sudan_pre_july_2011 = error_c_filtered_2[error_c_filtered_2['c_id'] == 59]
sudan_post_july_2011 = error_c_filtered_2[error_c_filtered_2['c_id'] == 245]
south_sudan_post_july_2011 = error_c_filtered_2[error_c_filtered_2['c_id'] == 246]

# Calculate summary statistics for the variable 'climate_pr'
summary_pre = sudan_pre_july_2011['climate_pr'].describe()
summary_post = sudan_post_july_2011['climate_pr'].describe()
summary_post_ssd = south_sudan_post_july_2011['climate_pr'].describe()

# Print the summaries
print("Summary statistics for 'climate_pr' before July 2011 (c_id = 59):\n", summary_pre)
print("\nSummary statistics for 'climate_pr' after July 2011 (c_id = 245):\n", summary_post)

# Optional: Check if the summaries are identical
identical = summary_pre.equals(summary_post)
print("\nAre the summary statistics for Sudan identical?", identical)

# Print the summary statistics for South Sudan
print("\nSummary statistics for 'climate_pr' for SOUTH SUDAN after July 2011 (c_id = 246):\n", summary_post_ssd)


#### Conclusion: Precipitation in the different territorial variations of Sudan
- The precipitation for Sudan is very similar in both periods.
- The precipitation is much higher in South Sudan in post-2011 compared to the post-2011 numbers from (North) Sudan.

- This suggests, that the SDN data is always exlusively for Northern Sudan, and that the data for South Sudan is separate.


--------

# Overall Conclusion: Country-Level Data Strcuture
- This suggests that the data is always structured along the current geographic specifications of global gountries today.
- Therefore, previous geographic configurations for countries, which have independent c_id's in VIEWS, are not covered by this data.

#### Problem
- Therefore, the current coding is likely incorrect.
    - The current coding suggests that pre 2011 data represents all of Sudan.
    - That is, because the iso3-country code "SDN" is coded as the c_id "59" before July 2011 and as "245" post July 2011.
    - However, that is incorrect. the c_id 59 represents all of Sudan. The data is however likely always restricted to represent Northern Sudan.
- This problem also extends to other countries. For example, Åland is coded as an independent country. By ommitting it, we lose all data on Åland, as the Åland data is not integrated into Finland data.


#### Potential Solutions
1) Approach 1: Ignore the issue and keep the data as is. Result: Wrong Sudan data for before 2011. => climate_variables_df_c_id_1
2) Approach 2: Omit pre-2011 data for "whole" Sudan. (Same for similar countries that had territorial changes.) => Most accurate adoption of data. Most missing data. = climate_variables_df_c_id_2
3) (Approach 3): Calculate pre-2011 data for Sudan on the basis of data from South and North Sudan. Something like the mean. But this would be very assumptive.
